In [1]:
from rest_module import make_rest_call

url = "http://amdex_server:8081/api/isJobAllowed/"
token = "abc123"

response = make_rest_call(url, token)
if response:
    print(response)
else:
    print("Error making REST API call")



Request headers: {'User-Agent': 'python-requests/2.28.2', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/json', 'Connection': 'keep-alive', 'Content-Length': '0', 'Authorization': 'Bearer abc123'}
{'allowed': True, 'userName': 'User 1', 'responseData': {'somedata': True}}
